# Import

In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import re
from app.utils.functions import *
from app.core import config
from IPython.display import HTML
from app.services.ollama_service import format_response
import httpx
from tqdm import tqdm
from tqdm.asyncio import tqdm

In [2]:
resids = await execute_sp(
    "dbo.sp_saResidences",
    {
        "user_fk": config.USER_FK,
    }
)
df_residences = pd.DataFrame(resids)
df_residences = df_residences[df_residences["sa_fk"].notna()][["sa_fk", "sa", "resp"]]
df_residences["sa_fk"] = df_residences["sa_fk"].astype(int)
# df_residences = df_residences[df_residences["sa"].apply(lambda x: len(str(x).split(" - ")[0]) > 4)]
sa_fks = df_residences["sa_fk"].to_list()
len(sa_fks)

227

# Data Preparation

In [3]:
res = await execute_sp(
    "dbo.sp_simBudLines",
    {
        "user_fk": config.USER_FK,
        "form_fk": 167,
        "line_fk": 0,
        "choix": 0,
        "isVisible": 1
    }
)
simple_dict = create_simplified_hierarchy(res)

In [31]:
async def data_preparation(sa_fk):
    data = await execute_sp(
        "ia.sp_simBudFormSA_one", 
        {
            "user_fk": config.USER_FK, 
            "sa_fk": sa_fk, 
            "form_fk": 167
        }
    )
    if data[0].get('EcrituresDetails') == None:
        return pd.DataFrame(None)
    json_string = data[0].get('EcrituresDetails')

    data_records = json.loads(json_string)
    context_data = pd.DataFrame(data_records)
    df = preprocessing_data(context_data, simple_dict)

    if df["Section  analytique"].unique().tolist() in [[''], [], None]:
        df["Section  analytique"] = df["Liste de sélection"]
        
    # Renommage et nettoyage
    df = df.rename(
        columns={
            'Code Hiérarchique': 'Code_H', 
            'Montant': 'Montant',
            'Lignes': 'Ligne_Analytique',
            'Contexte': 'Contexte',
            'Année': 'Annee',
            'Groupe': 'Groupe',
            'Section  analytique': 'Residence'
        }
    )

    df['Annee'] = df['Annee'].astype(int)
    df['Mois'] = df['Mois'].astype(int)
    df['Contexte'] = df['Contexte'].replace({'R': 'Réel', 'B': 'Budget', 'P': 'Prévision'})

    df_agg = df.groupby(
        [
            'Residence', 'Colonnes', 'Annee', 'Mois', "Nature de l'écriture", 'Contexte', 'Code_H', 'Ligne_Analytique', 'Groupe'
        ]
    )['Montant'].sum().reset_index()

    contexte_order = ['Réel', 'Prévision', 'Budget']

    def mois_sort_key(mois):
        try:
            return int(mois)
        except:
            return 99

    df_pivot = df_agg.pivot_table(
        index=['Groupe', 'Code_H', 'Ligne_Analytique'],
        columns=['Annee', 'Contexte', 'Mois', "Nature de l'écriture"],
        values='Montant',
        fill_value="N/A",
        aggfunc='sum'
    )
    # Explicitly infer objects to avoid FutureWarning from fill_value on object dtype
    df_pivot = df_pivot.infer_objects(copy=False)

    if df_pivot.columns.nlevels == 4:
        nature_unique = df_pivot.columns.get_level_values(3).unique().tolist()
        if "Annuelle" in nature_unique:
            nature_unique = [n for n in nature_unique if n != "Annuelle"]
            nature_order_desc = sorted(nature_unique, reverse=True) + ["Annuelle"]
        else:
            nature_order_desc = sorted(nature_unique, reverse=True)
        nature_order_dict = {name: i for i, name in enumerate(nature_order_desc)}
        
        def col_sort_key(x):
            return (
                int(x[0]) if str(x[0]).isdigit() else 0,
                contexte_order.index(x[1]) if x[1] in contexte_order else 99,
                mois_sort_key(x[2]),
                nature_order_dict.get(x[3], 999)
            )
        df_pivot = df_pivot[sorted(df_pivot.columns, key=col_sort_key)]

    df_pivot = df_pivot.reset_index()

    def code_hierarchical_sort_key(code):
        parts = [int(part) if part.isdigit() else part for part in re.split(r'\D+', str(code).strip('.')) if part]
        return parts

    df_pivot_sorted = df_pivot.copy()
    df_pivot_sorted['__sort_key'] = df_pivot_sorted['Code_H'].apply(code_hierarchical_sort_key)
    df_pivot_sorted = df_pivot_sorted.sort_values('__sort_key').drop(columns='__sort_key', level=0).reset_index(drop=True)
        
    mask_annuelle = df_pivot_sorted.columns.get_level_values(3) == "Annuelle"
    annuelle_cols = df_pivot_sorted.columns[mask_annuelle].tolist()

    meta_cols = [c for c in df_pivot_sorted.columns if c[0] in ("Groupe", "Code_H", "Ligne_Analytique")]

    selected_cols = meta_cols + annuelle_cols
    df_pivot_sorted_annual = df_pivot_sorted.loc[:, selected_cols]

    def format_value(val):
        try:
            if isinstance(val, str) and val.strip().startswith("%"):
                num = float(val.strip().replace("%", "").replace(",", "."))
                return "{:.2f} %".format(num)

            if isinstance(val, str) and "%" in val:
                num = float(val.replace("%", "").replace(",", ".").strip())
                return "{:.2f} %".format(num)

            if isinstance(val, (float, np.floating, int, np.integer)):
                if float(val) == int(val):
                    return int(val)
                else:
                    return "{:.2f}".format(float(val))

            if isinstance(val, str):
                num = float(val.replace(",", ".").strip())
                if num == int(num):
                    return int(num)
                else:
                    return "{:.2f}".format(num)
            return val
        except:
            return val

    for col in df_pivot_sorted_annual.columns:
        df_pivot_sorted_annual[col] = df_pivot_sorted_annual[col].apply(format_value)

    return df_pivot_sorted_annual

In [5]:
def select_value(df):
    current_year = datetime.now().year
    years = [current_year-3, current_year-2, current_year-1, current_year]

    df_h2_RR = df[
        df["Code_H"].apply(
            lambda x: (len(str(x).strip('.').split('.')) <= 1) and (str(x).strip('.').split('.')[0] in ['1', '2', '3', '4'])
        )
    ].reset_index(drop=True)

    meta_cols = df_h2_RR.columns[:3]
    year_cols = [col for col in df.columns[2:] if col[0] in years]
    cols_to_keep = list(meta_cols) + year_cols

    df_h2_RR_the_years = df_h2_RR.loc[:, cols_to_keep]
    
    years = [current_year-1, current_year]
    
    df_h3_CA = df[
        df["Code_H"].apply(
            lambda x: (len(str(x).strip('.').split('.')) <= 3) and (str(x).strip('.').split('.')[0] == '1')
        )
    ].reset_index(drop=True)

    meta_cols = df_h3_CA.columns[:3]
    year_cols = [col for col in df.columns[2:] if col[0] in years]
    cols_to_keep = list(meta_cols) + year_cols

    df_h3_the_years_CA = df_h3_CA.loc[:, cols_to_keep]

    df_h2_Charge_RR = df[
        df["Code_H"].apply(
            lambda x: (len(str(x).strip('.').split('.')) <= 2) and (str(x).strip('.').split('.')[0] in ['2'])
        )
    ].reset_index(drop=True)

    meta_cols = df_h2_Charge_RR.columns[:3]
    year_cols = [col for col in df.columns[2:] if col[0] in years]
    cols_to_keep = list(meta_cols) + year_cols

    df_h2_the_years_Charge_RR = df_h2_Charge_RR.loc[:, cols_to_keep]
    
    return df_h2_RR_the_years, df_h3_the_years_CA, df_h2_the_years_Charge_RR

# Ask Ollama

### Ollama Class

In [28]:
class OllamaClient:
    def __init__(self, ollama_url: str):
        self.client = httpx.AsyncClient(timeout=1000)
        self.url = ollama_url

    async def ask_ollama(self, prompt: str):
        payload = {
            "model": config.GPT,
            "messages": [
                {"role": "user", "content": prompt}
            ],
            "stream": False,
            "keep_alive": -1,
            "options": {
                "temperature": 0.1,
            }
        }

        response = await self.client.post(
            url=self.url, 
            json=payload
        )
        
        response.raise_for_status() 
        json_data = response.json() 
        if "message" in json_data and "content" in json_data["message"]: 
            content: str = json_data["message"]["content"] 
            parts = content.split("</think>", 1) 
            if len(parts) > 1: 
                content = parts[1] 
            else: 
                content = content 
            
            return content

ollamaClient = OllamaClient(config.OLLAMA_URL)

### To Markdown

In [ ]:
def transform_for_llm(df_pivot: pd.DataFrame):
    """
    Returns:
        - metric_text: str with metric blocks optimized for LLM input
    Notes:
        - This version detects if contextual columns (e.g. 'Groupe', 'Code_H', 'Ligne_Analytique')
            exist and builds a consolidated 'Indicateur' label from them.
        - Assumes helper functions _normalize_col and _format_euro_fr already exist.
    """

    def _format_euro_fr(x: float, line: str) -> str:
        """Format number in French style with 2 decimals and a non-breaking space thousands separator."""
        if pd.isna(x):
                return "N/A"
        elif str(line).startswith("%"):
            s = f"{x:,.2f}"
            s = s.replace(",", " ")
            return f"{s} %"
        else:
            s = f"{x:,.0f}"
            s = s.replace(",", " ")
            return f"{s} €"

    def _normalize_col(col):
        if isinstance(col, tuple):
            if len(col) >= 1:
                year = str(col[0])
            else:
                year = "unknown"
            # find Réel/Prévision/Budget if present in tuple (also accent-insensitive, lowercase!)
            typ = next(
                (str(x) for x in col if isinstance(x, str) and str(x).lower() in ("réel", "budget", "prévision", "prevision")),
                None
            )
            if typ is None:
                # search for any string in tuple as fallback
                typ = next((str(x) for x in col if isinstance(x, str)), "Réel")
            return {"year": year, "type": typ}
        else:
            # If col is not a tuple, fallback as string
            year = str(col)
            return {"year": year, "type": "Réel"}

    df = df_pivot.copy()

    # If index contains labels, reset to columns
    if df.index.name is None or df.index.name == "":
        df = df.reset_index()

    # If there are contextual columns, build a consolidated 'Indicateur' column
    context_cols = [c for c in ['Code_H', 'Ligne_Analytique', 'Indicateur'] if c in df.columns]

    if len(context_cols) > 1:
        # create a single descriptive indicator by joining available context columns (in order)
        df['Indicateur_consolide'] = df[context_cols].astype(str).apply(
            lambda row: " | ".join([str(x).strip() for x in row.values if str(x).strip() not in ['nan', 'None']]),
            axis=1
        )
        # prefer the consolidated name
        indicator_col = 'Indicateur_consolide'
    else:
        # detect a single indicator column if present, otherwise use first column
        indicator_col = None
        for possible in ['Ligne_Analytique', 'Indicateur', 'index', 0]:
            if possible in df.columns:
                indicator_col = possible
                break
        if indicator_col is None:
            indicator_col = df.columns[0]
        # if chosen indicator_col isn't already a string label, coerce to str
        if indicator_col != 'Indicateur':
            df[indicator_col] = df[indicator_col].astype(str)

    # Ensure the DataFrame has a column named exactly 'Indicateur' used downstream
    if indicator_col != 'Indicateur':
        df = df.rename(columns={indicator_col: 'Indicateur'})
    else:
        # if it already is 'Indicateur', ensure string type
        df['Indicateur'] = df['Indicateur'].astype(str)

    value_cols = [c for c in df.columns if c != 'Indicateur']

    rows = []
    for _, row in df.iterrows():
        # Avoid pandas row pretty-print for the indicator label
        if isinstance(row['Indicateur'], pd.Series):
            indicator_label = " | ".join(str(x).strip() for x in row['Indicateur'].values if str(x).strip() not in ['nan', 'None'])
        else:
            indicator_label = str(row['Indicateur']).strip()
        for col in value_cols:
            meta = _normalize_col(col)
            year = meta.get('year', 'unknown')
            typ = meta.get('type', 'Réel')
            try:
                val = row[col]
            except Exception:
                val = row.get(col, None)

            # Try to keep numeric
            numeric = None
            if pd.api.types.is_numeric_dtype(type(val)):
                try:
                    numeric = float(val) if not pd.isna(val) else None
                except Exception:
                    numeric = None
            else:
                try:
                    numeric = float(str(val).replace("€", "").replace("%", "").replace(" ", "").replace(",", "."))
                except Exception:
                    numeric = None

            lbl = indicator_label.split(" | ")[-1]
            txt = _format_euro_fr(numeric, lbl) if numeric is not None else "N/A"

            rows.append({
                'Indicateur': indicator_label,
                'Année': year,
                'Type': typ,
                'Valeur_num': numeric,
                'Valeur_txt': txt
            })

    df_long = pd.DataFrame(rows)

    # --- Custom LLM string format ---

    # Define context order mapping (for sorting)
    def context_rank(typ):
        t = str(typ).lower()
        if "réel" in t:
            return 0
        if "prevision" in t or "prévision" in t:
            return 1
        if "budget" in t:
            return 2
        return 99

    def _block_for_indicator(ind):
        label = str(ind).strip()
        lines = [f"[{label}]"]
        sub = df_long[df_long['Indicateur'].astype(str).values == str(ind)].copy()

        # Filter out technical garbage in 'Année'
        sub = sub[~sub['Année'].astype(str).str.lower().isin(['groupe', 'indicateur', 'index'])]

        # Attempt numeric year conversion for sorting; fallback keeps original order
        sub_sorted = sub.copy()
        try:
            sub_sorted["Année_num"] = pd.to_numeric(sub_sorted["Année"], errors='coerce')
        except Exception:
            sub_sorted["Année_num"] = sub_sorted["Année"]

        # First sort by Type context order, then by year
        if 'Type' in sub_sorted.columns:
            sub_sorted = sub_sorted.sort_values(
                by=["Type", "Année_num"],
                key=lambda col: col.map(context_rank) if col.name == "Type" else col,
                ascending=[True, True]
            )
        else:
            sub_sorted = sub_sorted.sort_values(by=["Année_num"])

        # Ensure ordering Réel -> Prévision -> Budget within each year
        entries = []
        for ctx in ["Réel", "Prévision", "Budget"]:
            sub_ctx = sub_sorted[sub_sorted["Type"].astype(str).str.lower().str.contains(ctx.lower(), na=False)]
            entries.append(sub_ctx)
        if entries:
            merged = pd.concat(entries)
            merged = merged.drop_duplicates(subset=["Année", "Type"])
        else:
            merged = sub_sorted

        # Produce lines
        for _, rr in merged.iterrows():
            year = rr['Année']
            typ = rr['Type']
            txt = rr['Valeur_txt']
            # Skip rows with completely empty or N/A values for non-year labels
            if (pd.isna(year) or str(year).strip().lower() in ['nan', 'none', '']) and txt in ["0", "0 €", "N/A"]:
                continue
            lines.append(f"- {year} {typ} : {txt}")
        return "\n".join(lines)

    indicators = df_long['Indicateur'].drop_duplicates().tolist()
    blocks = [_block_for_indicator(ind) for ind in indicators]

    metric_text = "\n\n".join(blocks)

    return metric_text


### Prompt

In [7]:
current_year = datetime.now().year

PROMPT_G = f"""
Tu es analyste financier senior spécialisé en exploitation de résidences étudiantes.

DONNÉES FOURNIES - TABLEAUX DE BORD OPÉRATIONNELS
{{}}

RÈGLES STRICTES
- Analyse uniquement à partir des données fournies.
- Aucune hypothèse, extrapolation ou interprétation externe.
- Valeur prioritaire : Réel > Prévision > Budget.

RÈGLE RÉEL / PRÉVISION
- Janvier à août : Prévision = Réel.
- Septembre à décembre : Prévision = projection du Réel.

AGRÉGATS ET FORMULES (DÉJÀ INTÉGRÉS)
- Marge 1 = Recettes - Charges d'immeuble directes.
- % DES RECETTES TOTALES = Marge / Recettes * 100.


ANALYSES OBLIGATOIRES
- Écart Réel/Prévision vs Budget ({current_year}).
- Évolution vs {current_year-1}.
- Analyse de la tendance {current_year-3}, {current_year-2}, {current_year-1} et {current_year}.
- Indicateur absent ou nul : écrire exactement « Non disponible ».

STYLE
- Français professionnel.
- Phrases courtes, factuelles.
- Toujours citer année et unité (€ ou %).
- TEXTE SIMPLE uniquement.

FORMAT DE SORTIE (STRICT)

**Résumé global** (4 lignes max)
- Dynamique générale (recettes vs charges).
- Écart majeur sur recettes ou charges (Dernière année).
- Lecture synthétique des marges.
- Appréciation opérationnelle.

**Risques principaux** (3 max)
- Valeur, année.

**Opportunités clés** (3 max)
- Valeur, année.

**Actions prioritaires** (3 max)
- Justification chiffrée.

**Conclusion de rentabilité** (En une phrase)
- Verdict clair : « Résidence rentable » ou « Résidence non rentable ».
- Justification chiffrée (marge, pourcentage des recettes, années).

CONTRAINTE FINALE
Respecte strictement la structure. Réponse concise.
"""

count_tokens(PROMPT_G)

320

In [8]:
PROMPT_RECETTES = f"""
Tu es analyste financier senior spécialisé en pilotage du chiffre d'affaires de résidences étudiantes.

DONNÉES FOURNIES - RECETTES
{{}}

RÈGLES STRICTES
- Aucune hypothèse ni extrapolation.
- Valeur prioritaire : Réel > Prévision > Budget.

RÈGLE RÉEL / PRÉVISION
- Janvier à août : Prévision = Réel.
- Septembre à décembre : Prévision = projection du Réel.

ANALYSES OBLIGATOIRES
- Écart Réel/Prévision vs Budget ({current_year}).
- Évolution {current_year} vs {current_year-1}.
- Valeur absente ou nulle : écrire « Non disponible ».

STYLE
- Français professionnel.
- Phrases courtes, factuelles.
- Années et unités obligatoires (€ / %).
- TEXTE SIMPLE uniquement.

FORMAT DE SORTIE (STRICT)

**Résumé recettes** (4 lignes max)
- Dynamique globale du chiffre d4affaires.
- Écart clé vs Budget ({current_year}).
- Variation vs {current_year-1}.
- Appréciation opérationnelle.

**Risques recettes** (3 max)
- Indicateur — valeur, année.

**Opportunités recettes** (3 max)
- Indicateur — valeur, année.

**Actions prioritaires** (3 max)
- Action — justification chiffrée.

CONTRAINTE FINALE
Respecte strictement la structure. Réponse concise.
"""
count_tokens(PROMPT_RECETTES)

225

In [9]:
PROMPT_CHARGES = f"""
Tu es analyste financier senior spécialisé en pilotage des charges d'exploitation de résidences étudiantes.

DONNÉES FOURNIES - CHARGES
{{}}

RÈGLES STRICTES
- Base-toi exclusivement sur les données fournies.
- Dernière valeur prioritaire : Réel > Prévision > Budget.

RÈGLE RÉEL / PRÉVISION
- Janvier à août : Prévision = Réel.
- Septembre à décembre : utiliser la Prévision.

ANALYSES OBLIGATOIRES
- Écart Réel/Prévision vs Budget ({current_year}).
- Évolution vs {current_year-1}.
- Valeur absente ou nulle : écrire exactement « Non disponible ».

STYLE
- Français professionnel.
- Phrases courtes.
- Toujours citer année et montant (€ ou %).
- TEXTE SIMPLE uniquement.

FORMAT DE SORTIE (STRICT)

**Résumé charges** (4 lignes max)
- Sens d'évolution global ({current_year}).
- Écart clé vs Budget.
- Poste le plus contributif.
- Appréciation opérationnelle.

**Postes sous surveillance** (3 max)
- Ecart (€), année.

**Opportunités d'optimisation** (3 max)
- Levier chiffré (€), année.

**Actions prioritaires** (3 max)
- Justification chiffrée.

CONTRAINTE FINALE
Respecte strictement la structure. Réponse concise.
"""

count_tokens(PROMPT_CHARGES)

232

### Pre Analyse

In [45]:
analyse_result = []

In [101]:
analyse_result = [{'sa_fk': 196,
  'global': '**Résumé global**  \n- Recettes 2025 1\u202f061\u202f875\u202f€ contre charges 338\u202f149\u202f€ → marge 723\u202f726\u202f€.  \n- Écart réel vs budget 2025 : recettes -29\u202f061\u202f€, marge -29\u202f845\u202f€.  \n- Marge 2025 723\u202f726\u202f€ > 2024 708\u202f602\u202f€ (+15\u202f124\u202f€).  \n- Opérationnellement rentable, mais marge légèrement en dessous du budget.\n\n**Risques principaux**  \n- Recettes 2025 vs Budget : -29\u202f061\u202f€  \n- Marge 2025 vs Budget : -29\u202f845\u202f€  \n- % Recettes 2025 vs Budget : -0,92\u202f%\n\n**Opportunités clés**  \n- Marge 2025 vs 2024 : +15\u202f124\u202f€  \n- % Recettes 2025 vs 2024 : +2,09\u202f%  \n- Charges 2025 vs 2024 : -25\u202f831\u202f€\n\n**Actions prioritaires**  \n- Augmenter recettes de 29\u202f061\u202f€ (≈2,7\u202f% de recettes) pour atteindre le budget.  \n- Réduire charges de 784\u202f€ (≈0,2\u202f% des charges) pour aligner marge.  \n- Améliorer services pour accroître marge de 29\u202f845\u202f€ (≈4,2\u202f% de marge).\n\n**Conclusion de rentabilité**  \nRésidence rentable : marge 723\u202f726\u202f€ et % recettes 68,16\u202f% en 2025, supérieurs aux seuils de rentabilité.',
  'recette': 'Non disponible',
  'charge': 'Non disponible',
  'time': '4mn 11s'},
 {'sa_fk': 8,
  'global': '**Résumé global**  \n- Recettes et charges augmentent chaque année, marge positive.  \n- En 2025, recettes en dessous du budget de 209\u202fk\u202f€ et charges supérieures de 151\u202fk\u202f€.  \n- Marge 2025 prévisionnelle : 10\u202f053\u202f249\u202f€ (72,25\u202f% des recettes).  \n- Résidence rentable, marge stable, pourcentage des recettes proche de 70\u202f%.  \n\n**Risques principaux**  \n- Recettes 2025 : -209\u202fk\u202f€  \n- Charges 2025 : +151\u202fk\u202f€  \n- % recettes 2025 : -1,49\u202f%  \n\n**Opportunités clés**  \n- Recettes 2024 : +1\u202f182\u202f051\u202f€  \n- Marge 2024 : +891\u202f949\u202f€  \n- Charges 2025 : +8\u202f314\u202f€ (croissance ralentie)  \n\n**Actions prioritaires**  \n- Réduire les charges 2025 de 151\u202fk\u202f€ pour atteindre le budget.  \n- Augmenter les recettes 2025 de 209\u202fk\u202f€ pour atteindre le budget.  \n- Maintenir la marge 2025 à 10\u202f414\u202f050\u202f€ (budget) en optimisant les coûts.  \n\n**Conclusion de rentabilité**  \nRésidence rentable : marge 2025 prévisionnelle de 10\u202f053\u202f249\u202f€ et pourcentage des recettes de 72,25\u202f%.',
  'recette': 'Non disponible',
  'charge': 'Non disponible',
  'time': '4mn 26s'},
 {'sa_fk': 189,
  'global': '**Résumé global**  \n- Recettes 2025 (+305\u202f334\u202f€ vs 2024) > charges (+134\u202f166\u202f€).  \n- Marge 2025 3\u202f952\u202f916\u202f€ (déclin de 168\u202f750\u202f€ par rapport à 2024).  \n- % recettes 2025 41,76\u202f% (baisse de 3,23\u202fpp).  \n- Opérationnelle : croissance des recettes, mais hausse des charges et marge en légère baisse.\n\n**Risques principaux**  \n- Charges 2025 > 2024 : +134\u202f166\u202f€  \n- Marge 2025 < 2024 : –168\u202f750\u202f€  \n- % recettes 2025 < 2024 : –3,23\u202fpp  \n\n**Opportunités clés**  \n- Recettes 2025 > 2024 : +305\u202f334\u202f€  \n- Marge 2024 la plus élevée : 4\u202f121\u202f666\u202f€  \n- % recettes 2024 la plus élevée après 2022 : 44,99\u202f%  \n\n**Actions prioritaires**  \n- Réduire charges 2025 de 134\u202f166\u202f€ (aligner sur budget).  \n- Accroître recettes 2025 de 305\u202f334\u202f€ (maintenir croissance).  \n- Optimiser marge 2025 de 168\u202f750\u202f€ (atteindre niveau 2024).  \n\n**Conclusion de rentabilité**  \nRésidence rentable – marge positive 3\u202f952\u202f916\u202f€ et % recettes 41,76\u202f% en 2025.',
  'recette': 'Non disponible',
  'charge': 'Non disponible',
  'time': '4mn 29s'},
 {'sa_fk': 183,
  'global': '**Résumé global**  \n- Recettes 2025 prévisionnelles 142\u202f107\u202f349\u202f€ vs 2024 157\u202f139\u202f784\u202f€ (‑15\u202f032\u202f435\u202f€).  \n- Charges 2025 83\u202f550\u202f651\u202f€ vs 2024 87\u202f273\u202f196\u202f€ (‑3\u202f722\u202f545\u202f€).  \n- Marge 2025 58\u202f556\u202f698\u202f€ vs 2024 69\u202f866\u202f588\u202f€ (‑11\u202f309\u202f890\u202f€).  \n- % recettes 2025 41,21\u202f% vs 2024 44,46\u202f% (‑3,25\u202fpp).  \n\n**Risques principaux**  \n- Recettes 2025 vs Budget : ‑21\u202f360\u202f151\u202f€  \n- Marge 2025 vs Budget : ‑18\u202f540\u202f244\u202f€  \n- % recettes 2025 vs Budget : ‑5,95\u202fpp  \n\n**Opportunités clés**  \n- Marge 2024 : 69\u202f866\u202f588\u202f€ (plus élevée)  \n- % recettes 2024 : 44,46\u202f% (plus élevé)  \n- Recettes 2023 : 147\u202f855\u202f957\u202f€ (croissance 2022‑2023)  \n\n**Actions prioritaires**  \n- Augmenter recettes 2025 de 21\u202f360\u202f151\u202f€ pour atteindre le budget.  \n- Réduire charges 2025 de 2\u202f819\u202f907\u202f€ pour améliorer la marge.  \n- Optimiser la rentabilité pour atteindre 77\u202f096\u202f942\u202f€ de marge 2025.  \n\n**Conclusion de rentabilité**  \nRésidence rentable – marge positive de 58\u202f556\u202f698\u202f€ et % recettes 41,21\u202f% en 2025.',
  'recette': 'Non disponible',
  'charge': 'Non disponible',
  'time': '3mn 51s'},
 {'sa_fk': 203,
  'global': '**Résumé global**  \n- Recettes ont progressé de 2022 à 2024, puis baissent en 2025 forecast.  \n- Écart majeur sur recettes\u202f: 2025 forecast 129\u202f392\u202f182\u202f€ vs 2024 144\u202f825\u202f668\u202f€ (‑15\u202f433\u202f486\u202f€).  \n- Marge 2025 forecast 55\u202f337\u202f677\u202f€ (‑11\u202f846\u202f008\u202f€ vs 2024).  \n- Opérationnellement, la résidence reste rentable mais doit corriger la baisse de recettes.  \n\n**Risques principaux**  \n- Recettes 2025 forecast 129\u202f392\u202f182\u202f€ (vs 2024 144\u202f825\u202f668\u202f€).  \n- Marge 2025 forecast 55\u202f337\u202f677\u202f€ (vs 2024 67\u202f183\u202f685\u202f€).  \n- % des recettes 2025 forecast 42,77\u202f% (vs 2024 46,39\u202f%).  \n\n**Opportunités clés**  \n- Marge 2024 67\u202f183\u202f685\u202f€ (pic).  \n- % des recettes 2024 46,39\u202f% (pic).  \n- Charges 2025 forecast 74\u202f054\u202f505\u202f€ (↓ vs 2024 77\u202f641\u202f983\u202f€).  \n\n**Actions prioritaires**  \n- Augmenter recettes de 15\u202f433\u202f486\u202f€ pour atteindre le niveau 2024.  \n- Réduire charges de 3\u202f587\u202f478\u202f€ pour aligner sur le forecast 2025.  \n- Améliorer % recettes de 3,62\u202fpp pour atteindre 46,39\u202f% (2024).  \n\n**Conclusion de rentabilité**  \nRésidence rentable\u202f: marge 55\u202f337\u202f677\u202f€ et % recettes 42,77\u202f% en 2025 forecast.',
  'recette': 'Non disponible',
  'charge': 'Non disponible',
  'time': '8mn 12s'},
 {'sa_fk': 195,
  'global': "**Résumé global**  \n- Recettes 2025 ont augmenté de 5,56\u202f% (9\u202f088\u202f987\u202f€) tandis que charges ont progressé de 10,05\u202f% (2\u202f943\u202f978\u202f€).  \n- Écart réel vs budget 2025 : recettes –177\u202f923\u202f€ ; charges –218\u202f305\u202f€.  \n- Marge 1 2025 : 6\u202f145\u202f009\u202f€ (67,61\u202f% des recettes).  \n- Tendance de croissance ralentit, marge en baisse de 1,32\u202f% des recettes.  \n\n**Risques principaux**  \n- Charges en hausse rapide (2025 vs 2024) : +10,05\u202f% (2\u202f943\u202f978\u202f€).  \n- % marge décroissant (2025 vs 2024) : –1,32\u202f% (67,61\u202f% vs 68,93\u202f%).  \n- Écart négatif vs budget (2025) : –218\u202f305\u202f€ charges.  \n\n**Opportunités clés**  \n- Croissance des recettes 2025 vs 2024 : +5,56\u202f% (9\u202f088\u202f987\u202f€).  \n- Marge 1 en hausse absolue : +209\u202f673\u202f€ (2025 vs 2024).  \n- Réduction de l'écart vs budget possible.  \n\n**Actions prioritaires**  \n- Réduire charges d'immeuble directes de 10\u202f% (objectif 2\u202f650\u202f000\u202f€) pour aligner budget.  \n- Augmenter revenus par optimisation de l'occupation de 3\u202f% (objectif 9\u202f400\u202f000\u202f€).  \n- Mettre en place suivi mensuel des marges pour corriger écarts.  \n\n**Conclusion de rentabilité**  \nRésidence rentable, marge positive de 6\u202f145\u202f009\u202f€ et 67,61\u202f% des recettes en 2025.",
  'recette': 'Non disponible',
  'charge': 'Non disponible',
  'time': '4mn 44s'},
 {'sa_fk': 184,
  'global': '**Résumé global**  \n- Recettes ont augmenté de 13\u202f668\u202f179\u202f€ de 2022 à 2024, puis ont chuté de 1\u202f156\u202f711\u202f€ en 2025.  \n- Charges ont grimpé de 4\u202f551\u202f065\u202f€ de 2022 à 2024, puis ont augmenté de 1\u202f830\u202f717\u202f€ en 2025.  \n- Marge 2025 réelle est de 98\u202f223\u202f967\u202f€, soit 2\u202f987\u202f428\u202f€ en baisse par rapport à 2024.  \n- La marge représente 67,61\u202f% des recettes en 2025, en baisse de 1,5\u202fpp par rapport à 2024.  \n\n**Risques principaux**  \n- Charges en hausse de 1\u202f830\u202f717\u202f€ (2025).  \n- Marge en baisse de 2\u202f987\u202f428\u202f€ (2025).  \n- Revenus en baisse de 1\u202f156\u202f711\u202f€ (2025).  \n\n**Opportunités clés**  \n- Réduction des charges de 1\u202f830\u202f717\u202f€ (2025).  \n- Augmentation des recettes de 1\u202f156\u202f711\u202f€ (2025).  \n- Amélioration de la marge de 2\u202f987\u202f428\u202f€ (2025).  \n\n**Actions prioritaires**  \n- Réduire charges de 1\u202f830\u202f717\u202f€ (2025) pour stabiliser la marge.  \n- Accroître recettes de 1\u202f156\u202f711\u202f€ (2025) via stratégies tarifaires.  \n- Optimiser la gestion des coûts pour récupérer 2\u202f987\u202f428\u202f€ de marge (2025).  \n\n**Conclusion de rentabilité**  \nRésidence rentable, marge de 98\u202f223\u202f967\u202f€ représentant 67,61\u202f% des recettes en 2025, malgré une baisse de 2\u202f987\u202f428\u202f€ par rapport à 2024.',
  'recette': 'Non disponible',
  'charge': 'Non disponible',
  'time': '6mn 25s'},
 {'sa_fk': 412,
  'global': '**Résumé global**  \n- Recettes augmentent de 3,80\u202f% en 2025 par rapport à 2024.  \n- Charges restent stables, hausse de 1,38\u202f% en 2025.  \n- Marge 1 passe de 14\u202f454\u202f449\u202f€ en 2024 à 15\u202f128\u202f037\u202f€ en 2025 (+4,66\u202f%).  \n- % recettes totales monte à 74,45\u202f% en 2025, meilleur niveau depuis 2022.\n\n**Risques principaux**  \n- Hausse des charges d’immeuble directes en 2023 (+10,92\u202f%).  \n- Baisse de la marge en 2023 (-5,58\u202f%).  \n- Variation de % recettes (2023 : -3,14\u202fpp).\n\n**Opportunités clés**  \n- Croissance des recettes en 2025 (+3,80\u202f%).  \n- Marge 1 en hausse de 4,66\u202f% en 2025.  \n- % recettes totales en hausse de 0,62\u202fpp en 2025.\n\n**Actions prioritaires**  \n- Optimiser les coûts d’entretien (réduction de 1,38\u202f% des charges).  \n- Accroître l’occupation pour soutenir la hausse des recettes (+3,80\u202f%).  \n- Renforcer la gestion financière pour maintenir la marge (+4,66\u202f%).\n\n**Conclusion de rentabilité**  \nRésidence rentable : marge 1 de 15\u202f128\u202f037\u202f€ en 2025, % recettes totales 74,45\u202f%.',
  'recette': 'Non disponible',
  'charge': 'Non disponible',
  'time': '4mn 30s'},
 {'sa_fk': 224,
  'global': '**Résumé global**  \n- Recettes ont augmenté de 10,9\u202f% en 2023 puis de 6,2\u202f% en 2024, stagnant en 2025.  \n- En 2025, charges dépassent le budget de 29\u202f280\u202f€ (vs 14\u202f291\u202f€ de recettes en dessous).  \n- Marge 2025 réelle de 1\u202f511\u202f678\u202f€ est 43\u202f571\u202f€ inférieure au budget.  \n- Opérationnellement, la résidence reste rentable mais la marge se contracte.\n\n**Risques principaux**  \n- Charges 2025 : +29\u202f280\u202f€ vs budget.  \n- Recettes 2025 : –14\u202f291\u202f€ vs budget.  \n- Marge 2025 : –43\u202f571\u202f€ vs budget.\n\n**Opportunités clés**  \n- Marge 2023 : +22,8\u202f% vs 2022.  \n- % recettes 2023 : +6,66\u202fpp vs 2022.  \n- Recettes 2023 : +10,9\u202f% vs 2022.\n\n**Actions prioritaires**  \n- Réduire charges 2025 de 10\u202f% (≈67\u202f990\u202f€) pour atteindre budget.  \n- Augmenter recettes 2025 de 5\u202f% (≈109\u202f579\u202f€) pour compenser déficit.  \n- Maintenir marge >1,5\u202fM\u202f€ en optimisant coûts indirects.\n\n**Conclusion de rentabilité**  \nRésidence rentable : marge 1\u202f511\u202f678\u202f€ en 2025, % recettes 68,98\u202f% > 60\u202f%.',
  'recette': 'Non disponible',
  'charge': 'Non disponible',
  'time': '4mn 28s'},
 {'sa_fk': 756,
  'global': '**Résumé global**  \n- Recettes 2023\u202f=\u202f3\u202f829\u202f€ vs 2022\u202f=\u202f4\u202f978\u202f€ (déclin\u202f=\u202f1\u202f149\u202f€).  \n- Marge 2023\u202f=\u202f3\u202f829\u202f€ =\u202f100\u202f% des recettes.  \n- Écart vs budget 2025\u202f=\u202fNon disponible.  \n- Rentabilité non évaluée sans données 2024‑2025.  \n\n**Risques principaux**  \n- Baisse des recettes 2023\u202f=\u202f3\u202f829\u202f€ (déclin\u202f=\u202f1\u202f149\u202f€).  \n- Marge 100\u202f% 2023\u202f=\u202f100\u202f% (absence de marge de sécurité).  \n- Écart vs budget 2025\u202f=\u202fNon disponible 2025.  \n\n**Opportunités clés**  \n- Augmenter recettes 2024\u202f=\u202fobjectif\u202f4\u202f500\u202f€ (≈\u202f+17\u202f%).  \n- Diversifier services 2024\u202f=\u202fobjectif\u202f+5\u202f% revenus.  \n- Réduire charges 2024\u202f=\u202fobjectif\u202f-10\u202f% coûts.  \n\n**Actions prioritaires**  \n- Analyser causes de baisse 2023\u202f=\u202fjustification\u202f1\u202f149\u202f€ de perte.  \n- Mettre en place plan d’augmentation 2024\u202f=\u202fobjectif\u202f+10\u202f% recettes.  \n- Réviser budget 2025\u202f=\u202fjustification\u202fNon disponible.  \n\n**Conclusion de rentabilité**  \nRésidence rentable – marge 3\u202f829\u202f€ (100\u202f% des recettes) en 2023.',
  'recette': 'Non disponible',
  'charge': 'Non disponible',
  'time': '4mn 36s'},
 {'sa_fk': 227,
  'global': '**Résumé global**  \n- Recettes ont baissé de 13,8\u202f% en 2025 par rapport à 2024.  \n- Charges ont diminué de 15,8\u202f% en 2025, mais restent élevées.  \n- Marge 2025 est de 959\u202f431\u202f€ (65,90\u202f% des recettes).  \n- Résidence reste rentable mais sous‑performance par rapport au budget.\n\n**Risques principaux**  \n- Recettes 2025 : -238\u202f309\u202f€  \n- Marge 2025 : -207\u202f221\u202f€  \n- Charges 2025 : -31\u202f088\u202f€\n\n**Opportunités clés**  \n- Marge 2023 vs 2022 : +57\u202f102\u202f€  \n- % des recettes 2025 vs 2024 : +0,82\u202fpp  \n- Charges 2025 vs 2024 : -93\u202f278\u202f€\n\n**Actions prioritaires**  \n- Augmenter les loyers 2025 de 200\u202f000\u202f€ (réduction de l’écart recettes).  \n- Réduire les charges 2025 de 20\u202f000\u202f€ (amélioration marge).  \n- Diversifier les revenus 2025 de 50\u202f000\u202f€ (stabilisation marge).\n\n**Conclusion de rentabilité**  \nRésidence rentable : marge positive de 959\u202f431\u202f€ et 65,90\u202f% des recettes en 2025.',
  'recette': 'Non disponible',
  'charge': 'Non disponible',
  'time': '4mn 25s'},
 {'sa_fk': 1137,
  'global': '**Résumé global**  \n- Recettes ont augmenté de 7\u202f% en 2023 puis de 13\u202f% en 2024 avant de chuter de 11\u202f% en 2025, tandis que les charges ont suivi un schéma similaire jusqu’en 2024 puis ont diminué de 14\u202f% en 2025.  \n- Écart réel/budget 2025 sur recettes : -10.97\u202f% (1\u202f968\u202f695\u202f€ vs 2\u202f211\u202f222\u202f€).  \n- Marge 2025 reste positive à 1\u202f139\u202f447\u202f€ (57.88\u202f% des recettes).  \n- Opérationnellement, la résidence conserve une rentabilité, mais la baisse des recettes 2025 soulève des incertitudes.\n\n**Risques principaux**  \n- Baisse des recettes 2025 (-11.20\u202f%)  \n- Baisse de marge 2025 (-8.90\u202f%)  \n- Écart réel/budget recettes 2025 (-10.97\u202f%)  \n\n**Opportunités clés**  \n- Hausse du % recettes 2025 (+1.46\u202fpp)  \n- Réduction des charges 2025 (-14.17\u202f%)  \n- Marge record 2024 (+18.53\u202f%)  \n\n**Actions prioritaires**  \n- Renforcer l’acquisition de locataires pour compenser la baisse recettes 2025 (-11.20\u202f%)  \n- Optimiser les coûts d’exploitation afin de stabiliser la marge 2025 (-8.90\u202f%)  \n- Réviser les prévisions 2025 pour aligner sur le budget et réduire l’écart -10.97\u202f%  \n\n**Conclusion de rentabilité**  \nRésidence rentable : marge positive 2025 €1\u202f139\u202f447, % recettes 57.88\u202f%.',
  'recette': 'Non disponible',
  'charge': 'Non disponible',
  'time': '5mn 26s'}]

In [17]:
def format_time_(time_delta: str):
    minutes, seconds = divmod(time_delta.total_seconds(), 60)
    return f"{int(minutes)}mn {int(seconds)}s"

In [18]:
async def analyse(sa_fk):
    st = datetime.now()

    df_pivot_sorted_annual = await data_preparation(sa_fk)
    if not df_pivot_sorted_annual.empty:
        df_h2_RR_the_years, df_h3_the_years_CA, df_h2_the_years_Charge_RR = select_value(df_pivot_sorted_annual)

        metric_text_g = transform_for_llm(df_h2_RR_the_years)
        # metric_text_ca = transform_for_llm(df_h3_the_years_CA)
        # metric_text_charge = transform_for_llm(df_h2_the_years_Charge_RR)

        brt_g = "Non disponible"
        brt_ca = "Non disponible"
        brt_charge = "Non disponible"

        try:
            brt_g = await ollamaClient.ask_ollama(PROMPT_G.format(metric_text_g))
        except Exception as e:
            print(e)
        # try:
        #     brt_ca = await ollamaClient.ask_ollama(PROMPT_RECETTES.format(metric_text_ca))
        # except:
        #     pass
        # try:
        #     brt_charge = await ollamaClient.ask_ollama(PROMPT_CHARGES.format(metric_text_charge))
        # except:
        #     pass

        time = datetime.now() - st

        return {
            "sa_fk": sa_fk,
            "global": brt_g,
            "recette": brt_ca,
            "charge": brt_charge,
            "time": format_time_(time)
        }
    return None

In [19]:
sa_fk_pres = []
for x in analyse_result:
    sa_fk_pres.append(x["sa_fk"])

async for sa_fk in tqdm(sa_fks):
    if sa_fk not in sa_fk_pres:
        ar = await analyse(sa_fk)
        if ar:
            analyse_result.append(ar)
            print("La résidence", ar["sa_fk"], df_residences[df_residences["sa_fk"] == sa_fk]["sa"].values[0], "est analysée en", ar["time"], ".")
        else:
            print("La résidence", sa_fk, df_residences[df_residences["sa_fk"] == sa_fk]["sa"].values[0], "n'est pas calculée.")

  1%|          | 2/227 [00:00<01:16,  2.94it/s]

name 'ollamaClient' is not defined
La résidence 196 AREF - Liste des ouvertures est analysée en 0mn 0s .


  1%|▏         | 3/227 [00:01<02:17,  1.63it/s]

name 'ollamaClient' is not defined
La résidence 8 AREF - Liste des résidences est analysée en 0mn 0s .


  2%|▏         | 4/227 [00:02<02:53,  1.29it/s]

...

La résidence 749 MARS01RG - Saint Jérome n'est pas calculée.
La résidence 1428 MARS02RG - Prado Castel n'est pas calculée.
La résidence 1693 MASS01RG - Massy Atlantis n'est pas calculée.
La résidence 750 ORLE01RG - Le Parc n'est pas calculée.


100%|██████████| 227/227 [00:22<00:00, 18.82it/s]

La résidence 751 TOUL01RG - Toulouse Brienne n'est pas calculée.


100%|██████████| 227/227 [00:23<00:00,  9.79it/s]

name 'ollamaClient' is not defined
La résidence 1137 VINC01RG - Vincennes est analysée en 0mn 0s .


### ---

In [105]:
current_year = datetime.now().year

SYSTEM_PROMPT = f"""
Tu es analyste financier senior spécialisé en exploitation de résidences étudiantes.

RÈGLES ABSOLUES
- Analyse strictement limitée aux données fournies par l'utilisateur.
- Aucune hypothèse, extrapolation ou connaissance externe.
- Priorité des valeurs : Réel > Prévision > Budget.
- Si une donnée est absente, écrire exactement : « Non disponible ».

CONTRAINTES MÉTIER
- Janvier-août : Prévision = Réel.
- Septembre-décembre : Prévision = projection du Réel.

STYLE ET SORTIE
- Langue : français professionnel.
- Phrases courtes et factuelles.
- Toujours citer l'année et l'unité (€ ou %).
- TEXTE SIMPLE uniquement. Aucun JSON, aucun code, aucune balise.
- Respect strict du format demandé. Aucune section supplémentaire.

MÉTHODOLOGIE
- Comparer systématiquement :
    1) Réel/Prévision {current_year} vs Budget {current_year}
    2) {current_year} vs {current_year-1}
- Les agrégats et formules sont considérés comme déjà corrects.
"""

In [106]:
USER_PROMPT_1 = f"""
DONNÉES - TABLEAUX DE BORD OPÉRATIONNELS (Résidence étudiante)

{{}}

ANALYSES À PRODUIRE OBLIGATOIREMENT
- Tendance {current_year-3} → {current_year}

FORMAT DE SORTIE (STRICT)

**Résumé global** (4 lignes max)
- Dynamique générale (recettes vs charges).
- Écart majeur sur la dernière année.
- Lecture synthétique des marges.
- Appréciation opérationnelle.

**Risques principaux** (3 max)
- Valeur, année.

**Actions prioritaires** (3 max)
- Justification chiffrée.

**Conclusion de rentabilité** (1 phrase)
- « Résidence rentable » ou « Résidence non rentable ».
- Justification chiffrée (marge, %, années).
"""

In [108]:
USER_PROMPT_2 = f"""
DONNÉES - TABLEAUX DE BORD OPÉRATIONNELS (Résidence étudiante) - RECETTES

{{}}

FORMAT DE SORTIE (STRICT)

**Résumé recettes** (4 lignes max)
- Dynamique globale du chiffre d'affaires.
- Écart clé vs Budget ({current_year}).
- Poste le plus contributif.
- Appréciation opérationnelle.

**Risques recettes** (3 max)
- Valeur, année.

**Actions prioritaires** (3 max)
- Justification chiffrée.
"""

In [109]:
USER_PROMPT_3 = f"""
DONNÉES FOURNIES - TABLEAUX DE BORD OPÉRATIONNELS (Résidence étudiante) - CHARGES

{{}}

STYLE
- Français professionnel.
- Phrases courtes.
- Toujours citer année et montant (€ ou %).
- TEXTE SIMPLE uniquement.

FORMAT DE SORTIE (STRICT)

**Résumé charges** (4 lignes max)
- Sens d'évolution global.
- Écart clé vs Budget ({current_year}).
- Poste le plus contributif.
- Appréciation opérationnelle.

**Postes sous surveillance** (3 max)
- Valeur, année.

**Actions prioritaires** (3 max)
- Justification chiffrée.
"""

In [43]:
async def get_prompt(sa_fk):

    df_pivot_sorted_annual = await data_preparation(sa_fk)
    if not df_pivot_sorted_annual.empty:
        df_h2_RR_the_years, df_h3_the_years_CA, df_h2_the_years_Charge = select_value(df_pivot_sorted_annual)

        try:
            metric_text_g = transform_for_llm(df_h2_RR_the_years)
            metric_text_r = transform_for_llm(df_h3_the_years_CA)
            metric_text_c = transform_for_llm(df_h2_the_years_Charge)
        except:
            return None, None, None

        return USER_PROMPT_1.format(metric_text_g), USER_PROMPT_2.format(metric_text_r), USER_PROMPT_3.format(metric_text_c)
    return None, None, None

In [21]:
import requests

In [97]:
def askLM(prompt):
    url = "http://si-5:1234/v1/chat/completions"
    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "model": "meta-llama-3.1-8b-instruct",
        "messages": [
            {
                "role": "system",
                "content": SYSTEM_PROMPT
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.3,
        "top_p": 0.9,
        "top_k": 40,
        "repeat_penalty": 1.1,
        "stream": False
    }

    response = requests.post(url, json=data, headers=headers)
    return response.json()['choices'][0]['message']['content']

In [117]:
async def analyse_v2(sa_fk):
    st = datetime.now()

    prompt_g, prompt_r, prompt_c = await get_prompt(sa_fk)
    
    brt_g = None
    brt_ca = None
    brt_charge = None
    
    if prompt_g is None or prompt_r is None or prompt_c is None:
        return None

    """ try:
        brt_g = askLM(prompt_g)
    except:
        pass """
    """ try:
        brt_ca = askLM(prompt_r)
    except:
        pass """

    try:
        brt_charge = askLM(prompt_c)
    except:
        pass
    
    time = datetime.now() - st

    return {
        "sa_fk": sa_fk,
        "global": brt_g,
        "recette": brt_ca,
        "charge": brt_charge,
        "time": format_time(time)
    }

In [118]:
analyse_result = []

In [123]:
sa_fk_pres = []
for x in analyse_result:
    # If 'x' is a coroutine, await it to fetch the actual result
    if hasattr(x, "__await__"):
        result = await x
    else:
        result = x
    sa_fk_pres.append(result["sa_fk"])

async for sa_fk in tqdm(sa_fks):
    if sa_fk not in sa_fk_pres:
        print(sa_fk, df_residences[df_residences["sa_fk"] == sa_fk]["sa"].values[0], "...")
        ar = await analyse_v2(sa_fk)
        if ar:
            analyse_result.append(ar)
            print("La résidence", ar["sa_fk"], df_residences[df_residences["sa_fk"] == sa_fk]["sa"].values[0], "est analysée en", ar["time"], ".")
        else:
            print("La résidence", sa_fk, df_residences[df_residences["sa_fk"] == sa_fk]["sa"].values[0], "n'est pas calculée.")

  4%|▍         | 10/227 [00:00<00:03, 62.55it/s]

185 Saint-Maur (STM01 + STM03) ...
La résidence 185 Saint-Maur (STM01 + STM03) n'est pas calculée.
401 19101099 - Région IDF Est ...
La résidence 401 19101099 - Région IDF Est n'est pas calculée.
1184 19101199 - Région IDF Ouest ...
La résidence 1184 19101199 - Région IDF Ouest n'est pas calculée.
1210 19101299 - Région direction Est ...


  7%|▋         | 17/227 [00:00<00:05, 35.48it/s]

La résidence 1210 19101299 - Région direction Est n'est pas calculée.
1741 19101399 - Région 2R 3R IDF ...
La résidence 1741 19101399 - Région 2R 3R IDF n'est pas calculée.
407 19102099 - Région Sud Ouest ...
La résidence 407 19102099 - Région Sud Ouest n'est pas calculée.
1185 19103199 - Région Rhone Alpes ...
La résidence 1185 19103199 - Région Rhone Alpes n'est pas calculée.
1186 19103299 - Région Centre ...
La résidence 1186 19103299 - Région Centre n'est pas calculée.
1214 19103399 - Région 4R IDF ...


 10%|▉         | 22/227 [00:00<00:07, 28.41it/s]

...

La résidence 2839 1TLS08RG - Estudines Toulouse Montaudran 2 n'est pas calculée.
952 1VLB01RG - Alliance ...
La résidence 952 1VLB01RG - Alliance n'est pas calculée.
1346 1VLC02RG - Valenciennes du Théâtre ...
La résidence 1346 1VLC02RG - Valenciennes du Théâtre n'est pas calculée.
917 1VLT01RG - Villetaneuse ...
La résidence 917 1VLT01RG - Villetaneuse n'est pas calculée.
2081 1VLZ01RG - Residence de Velizy PLS-ES ...
La résidence 2081 1VLZ01RG - Residence de Velizy PLS-ES n'est pas calculée.
1170 1VTY01RG - Vitry ...


 97%|█████████▋| 221/227 [00:10<00:00, 20.34it/s]

La résidence 1170 1VTY01RG - Vitry n'est pas calculée.
1246 1VTY02RG - Vitry Résidhome ...
La résidence 1246 1VTY02RG - Vitry Résidhome n'est pas calculée.
785 GREN01RG - Marie Curie ...
La résidence 785 GREN01RG - Marie Curie n'est pas calculée.
747 LYON01RG - Le Clip ...
La résidence 747 LYON01RG - Le Clip n'est pas calculée.
748 LYON02RG - Saxe Gambetta ...
La résidence 748 LYON02RG - Saxe Gambetta n'est pas calculée.
749 MARS01RG - Saint Jérome ...


100%|██████████| 227/227 [00:10<00:00, 20.71it/s]

La résidence 749 MARS01RG - Saint Jérome n'est pas calculée.
1428 MARS02RG - Prado Castel ...
La résidence 1428 MARS02RG - Prado Castel n'est pas calculée.
1693 MASS01RG - Massy Atlantis ...
La résidence 1693 MASS01RG - Massy Atlantis n'est pas calculée.
750 ORLE01RG - Le Parc ...
La résidence 750 ORLE01RG - Le Parc n'est pas calculée.
751 TOUL01RG - Toulouse Brienne ...
La résidence 751 TOUL01RG - Toulouse Brienne n'est pas calculée.


### Verification

In [124]:
print(len(sa_fk_pres))
for sa in sa_fk_pres:
    print(df_residences[df_residences["sa_fk"] == sa]["sa"].values[0])

11
AREF - Liste des ouvertures
AREF - Liste des résidences
REA - Liste des ouvertures
REA - Liste dynamique des résidences
REA - Test
RSG - Liste des ouvertures
RSG - Liste dynamique des résidences
19103099 - Région PACA
11000099 - Noisy
11400099 - Cergy Saint Christophe
VINC01RG - Vincennes


In [125]:
df = pd.DataFrame(analyse_result)
df["SA"] = df["sa_fk"].apply(lambda x: df_residences[df_residences["sa_fk"] == x]["sa"].values[0])
df = df[["sa_fk", "SA", "global", "recette", "charge", "time"]]
df.fillna("Non disponible")

,sa_fk,SA,global,recette,charge,time
0,196,AREF - Liste des ouvertures,Non disponible,Non disponible,**Résumé charges**\n\nLa charge totale 2024 Ré...,2mn 49s
1,8,AREF - Liste des résidences,Non disponible,Non disponible,**Résumé charges**\n\n- Les charges d'immeuble...,3mn 14s
2,189,REA - Liste des ouvertures,Non disponible,Non disponible,**Résumé charges**\n\nL'évolution des charges ...,3mn 2s
3,183,REA - Liste dynamique des résidences,Non disponible,Non disponible,**Résumé charges**\n\nLes charges d'immeuble d...,3mn 10s
4,203,REA - Test,Non disponible,Non disponible,**Résumé charges**\n\nLes charges d'immeuble d...,3mn 13s
5,195,RSG - Liste des ouvertures,Non disponible,Non disponible,**Résumé charges**\n\nLes charges d'immeuble d...,3mn 30s
6,184,RSG - Liste dynamique des résidences,Non disponible,Non disponible,**Résumé charges**\n\nLes charges d'immeuble d...,3mn 3s
7,412,19103099 - Région PACA,Non disponible,Non disponible,**Résumé charges**\n\n- L'évolution des charge...,3mn 0s
8,224,11000099 - Noisy,Non disponible,Non disponible,**Résumé charges**\n\nLes charges d'immeuble d...,2mn 41s
9,227,11400099 - Cergy Saint Christophe,Non disponible,Non disponible,**Résumé charges**\n\nLes charges d'immeuble d...,2mn 39s


In [126]:
charges = df["charge"].to_list()
charges

["**Résumé charges**\n\nLa charge totale 2024 Réel est de 363 980 € et celle de 2025 Prévision est de 338 149 €. L'écart clé vs Budget 2025 est de -1 784 €. Le poste le plus contributif en termes d'évolution négative est les Frais de personnel (-8,2 %). Une attention particulière doit être portée à la réduction des coûts.\n\n**Postes sous surveillance**\n\n- Frais de personnel : 98 254 € (2025 Prévision)\n- Énergie : -11,9 % (2025 Prévision vs 2024 Réel)\n\n**Actions prioritaires**\n\n1. Réduire les Frais de personnel de 8,2 % en 2025.\n2. Réduire les coûts d'énergie de 11,9 % en 2025.\n3. Réduire les coûts des Publicités Internet de 38,7 % en 2025.",
 "**Résumé charges**\n\n- Les charges d'immeuble directes augmentent de 0,6 % en 2025 par rapport à 2024.\n- L'écart clé vs Budget pour les charges totales est de -3,7 % en 2025 (3 861 125 € - 3 709 344 €).\n- Le poste le plus contributif en 2025 est les frais de personnel avec une prévision de 1 310 509 €.\n- L'évolution des charges est 

In [111]:
recettes = df["recette"].to_list()
recettes

["**Résumé recettes**\n\nLa dynamique globale du chiffre d'affaires en 2024 est de 1 072 581 €, soit une prévision pour 2025 de 1 061 875 €. L'écart clé par rapport au budget 2025 est de -2,9 % (1 061 875 € - 1 090 936 €). Le poste le plus contributif en 2024 est les loyers logements et parkings HT avec 1 056 602 €.\n\n**Risques recettes**\n\n- Risque : -3 090 €, Année : 2025 (Impayés).\n- Risque : -1 719 €, Année : 2025 (Commission Agence TO).\n- Risque : -936 €, Année : 2024 (Commission Agence TO).\n\n**Actions prioritaires**\n\n- Réduire les impayés en 2025 de -3 090 € pour atteindre le budget.\n- Améliorer la commission Agence TO en 2025 de -1 719 € pour atteindre le budget.\n- Augmenter les recettes annexes en 2025 de 12 520 € pour atteindre le budget.\n\nNote : Les chiffres sont exprimés en euros (€).",
 "**Résumé recettes**\n\n- La dynamique globale du chiffre d'affaires est positive entre 2024 et 2025 avec une prévision de 13 914 374 €, soit +1,7 % par rapport à 2024.\n- L'écar

In [88]:
globals = df["global"].to_list()
globals

["**Résumé global**\n\n- La dynamique générale est positive avec une augmentation des recettes et une diminution des charges.\n- L'écart majeur sur la dernière année concerne les marges avec un manque de 29 745 € en 2025 par rapport au budget.\n- Les marges ont augmenté de manière significative entre 2023 et 2024 (+496 852 €) mais sont inférieures à celles prévues pour 2025 (-30 845 €).\n- L'opération est rentable avec une marge moyenne positive sur les trois dernières années (708 602 € en 2024).\n\n**Risques principaux**\n\n- Risque de manque de recettes en 2025 avec un écart de -3 045 € entre le réel et la prévision.\n- Risque de charges directes trop élevées en 2025 avec une augmentation de +4 169 € par rapport à 2024.\n\n**Actions prioritaires**\n\n1. Réévaluer les prévisions de recettes pour 2025 afin d'atteindre le budget.\n2. Réduire les charges directes en 2025 pour atteindre la prévision.\n3. Augmenter les marges en 2025 en réduisant les coûts et en augmentant les revenus.\n\n

In [127]:
from datetime import timedelta

def parse_time_str(time_str):
    parts = time_str.strip().split(' ')
    minutes = seconds = 0
    for part in parts:
        if "mn" in part:
            minutes = int(part.replace("mn", ""))
        elif "s" in part:
            seconds = int(part.replace("s", ""))
    return timedelta(minutes=minutes, seconds=seconds)

mean_time = sum((parse_time_str(t) for t in df['time'].to_list()), timedelta())/len(df['time'].to_list())
print(str(mean_time))

0:03:05.727273


In [128]:
print(str(sum((parse_time_str(t) for t in df['time'].to_list()), timedelta())))

0:34:03


Analyse recette de *11 SA*, **0:59:45** au total et **0:05:25.909091** en moyenne.

Analyse charge de *11 SA*, **0:34:03** au total et **0:03:05.727273** en moyenne.

In [137]:
new_analyse_result = []

for i in range(len(analyse_result)):
    new_analyse_result.append({
        "sa_fk": analyse_result[i]["sa_fk"],
        "SA": df_residences[df_residences["sa_fk"] == analyse_result[i]["sa_fk"]]["sa"].values[0],
        "global": globals[i],
        "recette": recettes[i],
        "charge": charges[i],
        "time": analyse_result[i]["time"]
    })

pd.DataFrame(new_analyse_result)

,sa_fk,SA,global,recette,charge,time
0,196,AREF - Liste des ouvertures,**Résumé global**\n\n- La dynamique générale e...,**Résumé recettes**\n\nLa dynamique globale du...,**Résumé charges**\n\nLa charge totale 2024 Ré...,2mn 49s
1,8,AREF - Liste des résidences,**Résumé global**\n\n- La dynamique générale e...,**Résumé recettes**\n\n- La dynamique globale ...,**Résumé charges**\n\n- Les charges d'immeuble...,3mn 14s
2,189,REA - Liste des ouvertures,**Résumé global**\n\nLa dynamique générale est...,**Résumé recettes**\n\n- La dynamique globale ...,**Résumé charges**\n\nL'évolution des charges ...,3mn 2s
3,183,REA - Liste dynamique des résidences,**Résumé global**\n\n- La dynamique générale e...,**Résumé recettes**\n\n- La dynamique globale ...,**Résumé charges**\n\nLes charges d'immeuble d...,3mn 10s
4,203,REA - Test,**Résumé global**\n\n- La dynamique générale e...,**Résumé recettes**\n\n* Dynamique globale du ...,**Résumé charges**\n\nLes charges d'immeuble d...,3mn 13s
5,195,RSG - Liste des ouvertures,**Résumé global**\n\nLa dynamique générale est...,**Résumé recettes**\n\n- La dynamique globale ...,**Résumé charges**\n\nLes charges d'immeuble d...,3mn 30s
6,184,RSG - Liste dynamique des résidences,**Résumé global**\n\n- La dynamique générale e...,**Résumé recettes**\n\n- La dynamique globale ...,**Résumé charges**\n\nLes charges d'immeuble d...,3mn 3s
7,412,19103099 - Région PACA,**Résumé global**\n\n- La dynamique générale e...,**Résumé recettes**\n\n- La dynamique globale ...,**Résumé charges**\n\n- L'évolution des charge...,3mn 0s
8,224,11000099 - Noisy,**Résumé global**\n\nLa dynamique générale est...,**Résumé recettes**\n\n- La dynamique globale ...,**Résumé charges**\n\nLes charges d'immeuble d...,2mn 41s
9,227,11400099 - Cergy Saint Christophe,**Résumé global**\n\n- La dynamique générale e...,**Résumé recettes**\n\nLa dynamique globale du...,**Résumé charges**\n\nLes charges d'immeuble d...,2mn 39s


# BD

In [138]:
def replace_spaces_in_amount(text):
    return re.sub(r'(?<=\d) (?=\d|\€)', '\u202f', text)

async def to_dataBase(line: dict[str, str]):
    brt_g = replace_spaces_in_amount(line["global"]).replace("\u202f", "\\s") if line["global"] is not None else "Non disponible"
    brt_ca = replace_spaces_in_amount(line["recette"]).replace("\u202f", "\\s") if line["recette"] is not None else "Non disponible"
    brt_charge = replace_spaces_in_amount(line["charge"]).replace("\u202f", "\\s") if line["charge"] is not None else "Non disponible"

    res = await execute_sp(
        "dbo.sp_chatBotSAAnalyse_add",
        {
            "user_fk": config.USER_FK,
            "sa_fk": line["sa_fk"],
            "analyseGlobal": brt_g,
            "analyseRecette": brt_ca,
            "analyseCharge": brt_charge
        },
        config.DATABASE_URL_IA
    )

    print("SA", "->",df_residences[df_residences["sa_fk"] == line["sa_fk"]]["sa"].values[0])
    print("Analyse Global", "->", len(brt_g))
    print("Analyse Recette", "->", len(brt_ca))
    print("Analyse Charge", "->", len(brt_charge))
    print(res[0]["message"] if res else '')
    print("="*100)

In [135]:
len(new_analyse_result)

11

In [136]:
for line in new_analyse_result:
    if line["sa_fk"] not in []:
        await to_dataBase(line)

SA -> AREF - Liste des ouvertures
Analyse Global -> 1193
Analyse Recette -> 826
Analyse Charge -> 650
Enregistrement de l'analyse de la SA effectué avec succès
SA -> AREF - Liste des résidences
Analyse Global -> 1163
Analyse Recette -> 1631
Analyse Charge -> 921
Enregistrement de l'analyse de la SA effectué avec succès
SA -> REA - Liste des ouvertures
Analyse Global -> 1189
Analyse Recette -> 830
Analyse Charge -> 833
Enregistrement de l'analyse de la SA effectué avec succès
SA -> REA - Liste dynamique des résidences
Analyse Global -> 1424
Analyse Recette -> 2378
Analyse Charge -> 834
Enregistrement de l'analyse de la SA effectué avec succès
SA -> REA - Test
Analyse Global -> 1354
Analyse Recette -> 3130
Analyse Charge -> 798
Enregistrement de l'analyse de la SA effectué avec succès
SA -> RSG - Liste des ouvertures
Analyse Global -> 1529
Analyse Recette -> 1193
Analyse Charge -> 1039
Enregistrement de l'analyse de la SA effectué avec succès
SA -> RSG - Liste dynamique des résidences
An

In [83]:
content = df[df["sa_fk"] == 224]["global"].values[0]
content = format_response(content)
content = content.replace("\n", "")
display(HTML(content))

In [84]:
content

"<p><strong>Résumé global</strong></p><p>La dynamique générale est positive avec une augmentation des recettes et une stabilité des charges. L'écart majeur se situe en 2025 avec un écart entre le budget et la prévision de -1,4 %. Les marges sont en progression constante depuis 2022, atteignant 1 511 678 € en 2025.</p><p><strong>Risques principaux</strong></p><ul><li>Risque de sous-occupation en septembre-décembre 2025 : -13 291 €.</li><li>Risque de non-rentabilité en cas de dégradation des marges : 2024 = 1 529 494 €, 2023 = 1 452 222 €.</li></ul><p><strong>Actions prioritaires</strong></p><ul><li>Réduire les charges d'immeuble directes pour atteindre le budget de 650 619 € en 2025.</li><li>Augmenter les recettes en septembre-décembre 2025 pour atteindre la prévision de 2 191 577 €.</li><li>Améliorer la gestion des marges pour maintenir un niveau supérieur à 1 452 222 €.</li></ul><p><strong>Conclusion de rentabilité</strong><br>La résidence est considérée comme « Résidence rentable » e